In [1]:
import numpy as np
import pandas as pd

In [2]:
# 모듈이 없다는 오류가 발생해서 이 파일 한정으로 install 
!pip install lightgbm

# lightgbm을 사용하기 위해 불러오기
import lightgbm as lgb

# rmse수치도 확인해야 하기에 임포트
from sklearn.metrics import mean_squared_error

In [3]:
# 데이터 불러오기
df = pd.read_csv('./future_sales_final.csv')
df.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,main_type_id,sub_type_id,city_id,year,month,...,date_item_month_mean_lag_6,date_item_month_mean_lag_12,date_cat_month_mean_lag_1,date_shop_cat_month_mean_lag_1,date_shop_type_month_mean_lag_1,date_shop_subtype_month_mean_lag_1,date_city_month_mean_lag_1,date_city_item_month_mean_lag_1,date_type_month_mean_lag_1,date_subtype_month_mean_lag_1
0,0,5,5037,0,19,5,10,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,5,5320,0,55,13,2,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,5,5233,0,19,5,10,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,5,5232,0,23,5,16,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,5,5268,0,20,5,11,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'main_type_id', 'sub_type_id', 'city_id', 'year',
       'month', 'day', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'date_cnt_month_mean_lag_1', 'date_shop_month_mean_lag_1',
       'date_shop_month_mean_lag_2', 'date_shop_month_mean_lag_3',
       'date_shop_month_mean_lag_6', 'date_shop_month_mean_lag_12',
       'date_item_month_mean_lag_1', 'date_item_month_mean_lag_2',
       'date_item_month_mean_lag_3', 'date_item_month_mean_lag_6',
       'date_item_month_mean_lag_12', 'date_cat_month_mean_lag_1',
       'date_shop_cat_month_mean_lag_1', 'date_shop_type_month_mean_lag_1',
       'date_shop_subtype_month_mean_lag_1', 'date_city_month_mean_lag_1',
       'date_city_item_month_mean_lag_1', 'date_type_month_mean_lag_1',
       'date_subtype_month_mean_lag_1'],
      dtype='object')

In [6]:
# train, validation, test 데이터 3가지로 나누기

# 훈련 데이터 (피쳐)
X_train = df[df['date_block_num'] < 33]
X_train = X_train.drop(['item_cnt_month'], axis=1)

# 검증 데이터 (피처)
X_valid = df[df['date_block_num'] == 33]
X_valid = X_valid.drop(['item_cnt_month'], axis=1)

# 테스트 데이터 (피처)
X_test = df[df['date_block_num'] == 34]
X_test = X_test.drop(['item_cnt_month'], axis=1)

# 훈련 데이터 (타깃값)
y_train = df[df['date_block_num'] < 33]['item_cnt_month']
# 검증 데이터 (타깃값)
y_valid = df[df['date_block_num'] == 33]['item_cnt_month']



# 대회의 조건: item_cnt_month를 0~20으로 제한
# 따라서 훈련시킬 때 clip을 사용유무를 선택

# y_train = y_train.clip(0, 20)
# y_valid = y_valid.clip(0, 20)

# clip을 사용하지 않음
    # 조금이라도 더 정확한 값을 예측하고,
    # 추후에 데이터를 clip하는 게 맞다고 판단

In [ ]:
# 최적의 하이퍼파라미터 찾기
# Grid Search는 가능한 모든 조합을 시도하고 가장 좋은 조합을 선택하기 때문에
# Random Serach보다 오랜 시간 소요된다.
# Random Search는 무작위로 하이퍼파라미터를 선택하고 지정된 횟수만큼 탐색한다.
# Random Search는 Grid Search보다 계산 비용이 적지만, 최적의 하이퍼파라미터를 찾을 확률이 높아진다.

# Random Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
# Scientific Python.stats 모듈은 수치해석, 통계 등에 쓰이는 라이브러리 패키지
# 그중 randint 함수는 이 모듈에 포함된 랜덤 정수 생성 함수 중 하나이다.


# Random Search에 사용할 하이퍼파라미터 분포
param_dist = {
    'num_leaves': sp_randint(255, 1024),
    'learning_rate': [0.005, 0.01, 0.05],
    'feature_fraction': [0.75, 0.8, 0.85],
    'bagging_fraction': [0.75, 0.8, 0.85],
    'bagging_freq': [3, 5, 7],
    'force_col_wise': [True]
}

# Random Search 객체 생성
lgb_model = lgb.LGBMRegressor()
random_search = RandomizedSearchCV(lgb_model, param_distributions=param_dist,
                                   cv=3, scoring='neg_root_mean_squared_error',
                                   verbose=2, n_jobs=4, n_iter=50)

# Random Search 수행
random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 평가 점수 출력
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.85, subsample=1.0 will be ignored. Current value: bagging_fraction=0.85
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Best parameters: {'bagging_fraction': 0.85, 'bagging_freq': 5, 'feature_fraction': 0.8, 'force_col_wise': True, 'learning_rate': 0.05, 'num_leaves': 282}
Best score: -0.7504977657365307


- num_leaves: 의사 결정 트리의 잎 노드 개수
    - 너무 크면 과적합을 유발하고, 너무 작으면 과소적합이 발생할 수 있습니다.
    - 따라서 255부터 1024사이의 정수값 중에서 랜덤으로 선택한다는 것을 의미합니다.
    - 이 범위로 설정하는 게 가장 일반적이라고 해서 이렇게 설정하였습니다.
- learning_rate: 학습 속도
- feature_fraction: 각 트리를 학습할 때 무작위로 선택되는 특성의 비율
    - 이를 통해 각 트리가 서로 다른 특성에 대해 학습하도록 유도하여 모델의 다양성을 높일 수 있습니다.
- bagging_fraction: 설정한 값만큼의 샘플이 무작위로 선택한다는 것을 의미
    - 디폴트값은 1이지만, 모델의 복잡도를 줄이기 위해 일반적으로 0.5~0.8을 사용합니다.
    - 하지만 값이 너무 낮으면 모델 성능이 저하될 수 있으므로 적절한 값을 조정해야 합니다.
- bagging_freq: bagging이 적용될 빈도수
    - 일반적으로 작은 값을 선택합니다.
- force_col_wise: 컬럼 단위로 계산하는 것을 의미(True or False)
    - Light GBM은 컬럼 중심으로 구현되어 있기 때문에, 이걸 True로 설정하면 더 빠르게 계산할 수 있습니다.

In [21]:
# Best parameters: {'bagging_fraction': 0.85, 'bagging_freq': 5,
#                   'feature_fraction': 0.8, 'force_col_wise': True,
#                   'learning_rate': 0.05, 'num_leaves': 282}
# Best score: -0.7504977657365307

<!-- Best parameters: {'bagging_fraction': 0.85, 'bagging_freq': 5,
                  'feature_fraction': 0.8, 'force_col_wise': True,
                  'learning_rate': 0.05, 'num_leaves': 282}
Best score: -0.7504977657365307 -->

In [18]:
# Random Search의 결과값에 맞춘 파라미터 설정


# LightGBM 하이퍼파라미터
params = {'metric': 'rmse',
          'bagging_fraction': 0.85,
          'bagging_freq': 5,
          'feature_fraction': 0.8,
          'force_col_wise': True,
          'learning_rate': 0.05,
          'num_leaves': 282}

cat_features = ['shop_id', 'city_id', 'item_category_id', 'main_type_id', 'month']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)

# 결과값이 training's rmse: 0.675387	valid_1's rmse: 0.863069으로, 과적합으로 예측됨

c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['city_id', 'item_category_id', 'main_type_id', 'month', 'shop_id']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM

[LightGBM] [Info] Total Bins 5097
[LightGBM] [Info] Number of data points in the train set: 7068600, number of used features: 34
[LightGBM] [Info] Start training from score 0.185447


c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 0.657033	valid_1's rmse: 0.86517
[200]	training's rmse: 0.622237	valid_1's rmse: 0.869203
Early stopping, best iteration is:
[70]	training's rmse: 0.675387	valid_1's rmse: 0.863069


1. 첫 번째 조정
```
params = {'metric': 'rmse', 
          'num_leaves': 31,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'force_col_wise': True,
          'random_state': 10}
training's rmse: 0.726692	valid_1's rmse: 0.872315
```


2. 두 번째 조정
```
params = {'metric': 'rmse', 
          'num_leaves': 31,
          'learning_rate': 0.005,
          'feature_fraction': 0.8, # 이 부분을 조정함
          'bagging_fraction': 0.7,
          'force_col_wise': True,
          'random_state': 10}
training's rmse: 0.757593	valid_1's rmse: 0.87795
```
과적합에서 조금 벗어났음을 확인함

In [22]:
# 디폴트 (최종)

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse', 
          'num_leaves': 31,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 42}

cat_features = ['shop_id', 'city_id', 'item_category_id', 'main_type_id', 'month']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)

[LightGBM] [Info] Total Bins 5093
[LightGBM] [Info] Number of data points in the train set: 7068600, number of used features: 34
[LightGBM] [Info] Start training from score 0.185447
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 0.880911	valid_1's rmse: 0.955225
[200]	training's rmse: 0.809135	valid_1's rmse: 0.904068
[300]	training's rmse: 0.775133	valid_1's rmse: 0.884246
[400]	training's rmse: 0.757705	valid_1's rmse: 0.877205
[500]	training's rmse: 0.747897	valid_1's rmse: 0.874085
[600]	training's rmse: 0.741328	valid_1's rmse: 0.87268
[700]	training's rmse: 0.736637	valid_1's rmse: 0.871901
[800]	training's rmse: 0.733022	valid_1's rmse: 0.871489
[900]	training's rmse: 0.730092	valid_1's rmse: 0.871311
[1000]	training's rmse: 0.727413	valid_1's rmse: 0.871418
Early stopping, best iteration is:
[850]	training's rmse: 0.731488	valid_1's rmse: 0.871237


In [32]:
# 디폴트 (최종)

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse', 
          'num_leaves': 31,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 42}

cat_features = ['shop_id', 'city_id', 'item_category_id', 'main_type_id', 'month']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      verbose_eval=100)

c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['city_id', 'item_category_id', 'main_type_id', 'month', 'shop_id']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 5093
[LightGBM] [Info] Number of data points in the train set: 7068600, number of used features: 34
[LightGBM] [Info] Start training from score 0.185447


c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\SBAUser\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	training's rmse: 0.880911	valid_1's rmse: 0.955225
[200]	training's rmse: 0.809135	valid_1's rmse: 0.904068
[300]	training's rmse: 0.775133	valid_1's rmse: 0.884246
[400]	training's rmse: 0.757705	valid_1's rmse: 0.877205
[500]	training's rmse: 0.747897	valid_1's rmse: 0.874085
[600]	training's rmse: 0.741328	valid_1's rmse: 0.87268
[700]	training's rmse: 0.736637	valid_1's rmse: 0.871901
[800]	training's rmse: 0.733022	valid_1's rmse: 0.871489
[900]	training's rmse: 0.730092	valid_1's rmse: 0.871311
[1000]	training's rmse: 0.727413	valid_1's rmse: 0.871418
[1100]	training's rmse: 0.725109	valid_1's rmse: 0.871349
[1200]	training's rmse: 0.723082	valid_1's rmse: 0.871158
[1300]	training's rmse: 0.721063	valid_1's rmse: 0.871307
[1400]	training's rmse: 0.718931	valid_1's rmse: 0.87104
[1500]	training's rmse: 0.71708	valid_1's rmse: 0.870725


In [26]:
preds = lgb_model.predict(X_test).clip(0,20)
preds

array([0.70854574, 0.04817206, 1.12395567, ..., 0.05909546, 0.03353667,
       0.06965296])

In [27]:
submission = pd.read_csv('./sample_submission.csv')
submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [28]:
submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)